# client instance 생성

In [1]:
DEPLOYMENT_URL="http://10.8.1.229:8000/"
DEPLOYMENT_URL="http://localhost:2024/"

from langgraph_sdk import get_client

client = get_client(url=DEPLOYMENT_URL)

# thread 생성 (앨런의 channel에 대응)

- 생성 요청 후 thread_id를 포함한 데이터를 return 받음
- 하나의 thread에 대해서 여러 에이전트를 호출하고 대화가능
- 현재 앨런은 한개의 channel에 대해서 한개의 agent가 1:1로 대응되므로 현재는 고려할 필요 없지만, 추후에 1개의 채널에 대해서 여러 agent를 호출가능

In [2]:
thread = await client.threads.create()
print(thread)

thread_id = thread["thread_id"]
print(thread_id)

{'thread_id': '24159d9a-62ca-471d-acb3-3f1a21f52c7b', 'created_at': '2025-08-13T00:58:56.323214+00:00', 'updated_at': '2025-08-13T00:58:56.323216+00:00', 'metadata': {}, 'status': 'idle', 'config': {}, 'values': None}
24159d9a-62ca-471d-acb3-3f1a21f52c7b


# 에이전트 호출

- 해당 thread에서 어떤 agent를 호출할 지 assistant_id를 포함하여 요청을 보냄

## parameters
- thread_id : 위에서 생성한 thread_id
- assistant_id : 앨런의 persona_id에 대응, agent를 구분하는 id, 슬라이드 에이전트의 경우 "slide_generate_agent"
- input : 해당 agent 요청에 대한 parameter, 현재는 messages만 존재, 앨런 페이지에서 유저로 부터 받는 input을 해당 parameter에 dict로 넣어줌
- stream_mode : 어떤 event에 따라서 메시지를 받을지 지정, updates(한 단계가 끝날 때마나 메시지가 옴), events(llm이 생성하는 단위로 메시지 생성)

In [3]:
list_msg_id = list()


def get_data(chunk):
    data = list(chunk.data.values())[0]
    return data


def filter_ai_msg(chunk):
    # 유저와 채팅 창에 출력될 ai 메시지를 필터링하는 함수
    if 'updates' in chunk.event:
        data = get_data(chunk)

        if data is not None:
            if 'messages' in data.keys():
                if data['messages'][-1]["name"] is not None:
                    msg_type = data['messages'][-1]['type']
                    msg_id = data['messages'][-1]['id']
                    if msg_id not in list_msg_id and msg_type == "ai":
                        list_msg_id.append(msg_id)
                        return True
    return False


def filter_silde_data(chunk):
    # slide 관련 데이터를 담은 메시지를 필터링하는 함수
    # 슬라이드 한페이지 단위로 메시지가 옴
    if 'updates' in chunk.event:
        node_name = chunk.data.keys()
        if 'executor' in node_name:
            return True

In [4]:
assistant_id = "slide_generate_agent"
list_chunk = list()

In [5]:
input = { "messages": "안녕"}

async for chunk in client.runs.stream(
    thread_id,
    assistant_id=assistant_id,
    input=input,
    stream_mode=["updates"],
    stream_subgraphs=True
):
    try:
        list_chunk.append(chunk)
        if filter_ai_msg(chunk):
            print("#" * 10 + "  AI 메시지  " + "#" * 10)
            print(chunk.data['agent']['messages'][-1]['content'])
        elif filter_silde_data(chunk):
            print("#" * 10 + "  슬라이드 데이터  " + "#" * 10)
            print(chunk)
    except:
        continue
    pass

##########  AI 메시지  ##########
안녕하세요! 슬라이드를 만드는 데 도움을 드릴 수 있어요. 어떤 주제의 슬라이드를 만들고 싶으신가요?


In [6]:
input = { "messages": "제주도 여행을 주제로 슬라이드를 만들어줘"}

async for chunk in client.runs.stream(
    thread_id,
    assistant_id=assistant_id,
    input=input,
    stream_mode=["updates"],
    stream_subgraphs=True
):
    try:
        list_chunk.append(chunk)
        if filter_ai_msg(chunk):
            print("#" * 10 + "  AI 메시지  " + "#" * 10)
            data = get_data(chunk)
            print(data['messages'][-1]['content'])
        elif filter_silde_data(chunk):
            print("#" * 10 + "  슬라이드 데이터  " + "#" * 10)
            print(chunk)
    except:
        continue
    pass

##########  AI 메시지  ##########
좋습니다! 제주도 여행에 관한 슬라이드를 만들기 위해 몇 가지 질문을 드리겠습니다.

1. 슬라이드의 목적은 무엇인가요? (예: 여행 계획, 소개, 추천 코스 등)
2. 어떤 정보를 포함하고 싶으신가요? (예: 관광 명소, 맛집 정보, 교통편, 여행 일정 등)
3. 대상 청중은 누구인가요? (예: 가족 여행, 친구와의 여행, 개인 여행 등)
4. 슬라이드에 포함할 페이지 수나 특정 레이아웃이 필요한가요?
5. 추가하고 싶은 이미지나 그래픽 요소가 있나요?


In [7]:
input = {"messages": "알아서 해줘"}

async for chunk in client.runs.stream(
    thread_id,
    assistant_id=assistant_id,
    input=input,
    stream_mode=["updates"],
    stream_subgraphs=True
):

    if filter_ai_msg(chunk):
        print("#" * 10 + "  AI 메시지  " + "#" * 10)
        data = get_data(chunk)
        msg_id = data['messages'][-1]['id']

        print(data['messages'][-1]['content'])
    elif filter_silde_data(chunk):
        print("#" * 10 + "  슬라이드 데이터  " + "#" * 10)


##########  AI 메시지  ##########

##########  AI 메시지  ##########
검색 도구를 사용하여 목차를 생성하기 위한 조사를 시작합니다.
##########  AI 메시지  ##########
생성된 목차는 다음과 같습니다. 

2. 제주도 여행 - 제주도의 주요 관광 명소를 소개하고, 방문해야 할 필수 장소들을 탐방하는 코너입니다. 
3. 제주도 여행 - 제주도에서 꼭 맛봐야 할 향토 음식을 소개하고, 추천 맛집 정보를 제공합니다. 
4. 제주도 여행 - 제주도에서 즐길 수 있는 다양한 액티비티와 체험 프로그램을 안내합니다. 
5. 제주도 여행 - 제주도 여행을 보다 효율적이고 쉽게 할 수 있는 팁과 가이드를 제공합니다. 
6. 제주도 여행 - 제주도 현지 교통수단과 렌터카 이용에 관한 정보를 제공합니다. 

##########  슬라이드 데이터  ##########
##########  슬라이드 데이터  ##########
##########  슬라이드 데이터  ##########
##########  슬라이드 데이터  ##########
##########  슬라이드 데이터  ##########
##########  슬라이드 데이터  ##########
##########  슬라이드 데이터  ##########
##########  AI 메시지  ##########
이제 제주도 여행에 대한 슬라이드를 만들 준비가 되었습니다. 위의 목차를 기반으로 슬라이드를 생성할 것입니다. 추가적인 요청이 있으면 언제든지 말씀해 주세요!


In [8]:
for chunk in list_chunk:
    if filter_silde_data(chunk):
        print("#" * 10 + "  슬라이드 데이터  " + "#" * 10)
        print(chunk.data)
        break

In [9]:
for chunk in list_chunk:
    if filter_ai_msg(chunk):
        print("#" * 10 + "  AI 메시지  " + "#" * 10)
        data = get_data(chunk)
        print(data['messages'][-1]['content'])


In [10]:
state = await client.threads.get_state(thread_id=thread_id)

for msg in state["values"]["messages"]:
    if msg["type"] != "human":
        print("#" * 10 + "  AI 메시지  " + "#" * 10)
        print(msg["content"])

##########  AI 메시지  ##########
안녕하세요! 슬라이드를 만드는 데 도움을 드릴 수 있어요. 어떤 주제의 슬라이드를 만들고 싶으신가요?
##########  AI 메시지  ##########
좋습니다! 제주도 여행에 관한 슬라이드를 만들기 위해 몇 가지 질문을 드리겠습니다.

1. 슬라이드의 목적은 무엇인가요? (예: 여행 계획, 소개, 추천 코스 등)
2. 어떤 정보를 포함하고 싶으신가요? (예: 관광 명소, 맛집 정보, 교통편, 여행 일정 등)
3. 대상 청중은 누구인가요? (예: 가족 여행, 친구와의 여행, 개인 여행 등)
4. 슬라이드에 포함할 페이지 수나 특정 레이아웃이 필요한가요?
5. 추가하고 싶은 이미지나 그래픽 요소가 있나요?
##########  AI 메시지  ##########

##########  AI 메시지  ##########
Successfully transferred to slide_generate_agent
##########  AI 메시지  ##########
검색 도구를 사용하여 목차를 생성하기 위한 조사를 시작합니다.
##########  AI 메시지  ##########
생성된 목차는 다음과 같습니다. 

2. 제주도 여행 - 제주도의 주요 관광 명소를 소개하고, 방문해야 할 필수 장소들을 탐방하는 코너입니다. 
3. 제주도 여행 - 제주도에서 꼭 맛봐야 할 향토 음식을 소개하고, 추천 맛집 정보를 제공합니다. 
4. 제주도 여행 - 제주도에서 즐길 수 있는 다양한 액티비티와 체험 프로그램을 안내합니다. 
5. 제주도 여행 - 제주도 여행을 보다 효율적이고 쉽게 할 수 있는 팁과 가이드를 제공합니다. 
6. 제주도 여행 - 제주도 현지 교통수단과 렌터카 이용에 관한 정보를 제공합니다. 

##########  AI 메시지  ##########
Transferring back to supervisor
##########  AI 메시지  ##########
Successfully transferred b

In [11]:
len(state["values"]["slides"])

14

In [12]:
state["values"]["slides"]

[{'slide_type': 'title',
  'topic': '제주도 여행',
  'idx': 0,
  'name': '타이틀 슬라이드',
  'description': '제주도 여행에 대한 프레젠테이션 타이틀 페이지',
  'requirements': ['타이틀 텍스트', '부제목 또는 설명'],
  'research': True,
  'content': "## 타이틀 슬라이드\n\n제주도(Jeju Island)는 한국의 남부 해안에 위치한 화산섬으로, 매력적인 관광지로 잘 알려져 있습니다. 제주도는 유네스코가 지정한 세계자연유산으로 다양한 자연 경관과 고유의 생태계를 자랑합니다. 여기에는 성산일출봉(Seongsan Ilchulbong), 한라산(Hallasan Mountain), 그리고 만장굴(Manjanggul Cave) 같은 독특한 명소들이 포함됩니다[1][2]. \n\n이곳은 한국뿐만 아니라 세계 여러 나라의 관광객들에게도 인기 있는 여행지입니다. 자연경관뿐만 아니라 제주도는 한류 문화의 중심지로, 많은 한국 드라마의 촬영지로도 유명합니다[3]. 또한, 제주도에서는 해녀(Haenyeo)의 독특한 문화도 경험할 수 있어 특별한 여행 경험을 제공합니다[4].\n\n제주도 여행 프레젠테이션의 타이틀 슬라이드는 제주도의 자연적 아름다움과 문화적 매력을 강조하며, 다양한 관광 명소와 활동을 시각적으로 표현할 필요가 있습니다. 제목은 '제주도 여행: 천혜의 자연과 문화 탐방'으로 설정할 수 있으며, 제주도의 주요 명소 그림이나 영상으로 슬라이드를 장식하여 청중의 관심을 끌어야 합니다. 이 타이틀 페이지는 앞으로의 프레젠테이션이 다룰 다양한 주제에 대한 짧은 소개를 포함하여, 전체적인 결론과 함게 제주도가 제공하는 특별한 경험을 효과적으로 전달해야 합니다.\n\n### Sources\n[1] Jeju Island Special Tourist Zone (제주도 관광특구) - Visit Korea: https://english.visitkorea.or.kr/svc